# Interest Rate Model Calibration

In this notebook, we analyse the calibration of 3-factor Gaussian HJM models from the [DiffFusion.jl](https://github.com/frame-consulting/DiffFusion.jl) framework.

Calibration objectives are statistical properties of reference interest rates. The targeted statistical properties are volatilities and correlations of 1-month returns observed from a historical time series spanning 20 years. The data generation procedure and calculation methodology for reference volatilities and correlations are specified in the related [MarketData](https://github.com/sschlenkrich/MarketData) repository.

The calibrated model is considered a *real-world* model. The model will recover observed statistical properties. But the resulting model will a priori *not* match prices for interest rate options observed in the market at a given date. Nevertheless, the calibrated model is intended to provide a starting point (or initial guess) for risk neutral calibration to interest rate option data.

An objective of the real-world calibration is to mark model parameters which are difficult to calibrate to option data. Such model parameters are mean reversion rates and instantaneous risk factor correlations.

We note that the proposed model parameter calibration does not affect the Gaussian HJM model's ability to match the initial yield curve observed from market data. Our calibration methodology is formulated such that it does not depend on model input yield curves.

## Packages and Preliminaries

In [ ]:
using Pkg
Pkg.activate("../.")

In [ ]:
using CSV
using DataFrames
using DiffFusion
using ForwardDiff
using PlotlyJS

In [ ]:
include("utils.jl")

## Reference Rate Specification

Reference interest rates are continuously compounded zero rates.

$$
z_{t}\left(\tau\right)=-\frac{\log\left[P\left(t,t+\tau\right)\right]}{\tau}.
$$

Properties of model-implied zero rates for yield curve modelling are discussed e.g. in [Real-World Yield Curve Shape Modelling with Multi-Factor HJM Models](https://ssrn.com/abstract=4638188).

We use rate terms $\tau \in \{ 1, 2, 5, 10, 15, 20 \}$ (measured in years).

## Reference Rate Data

Calibration targets are estimated from historical time series. The targets are represented by ranges for volatilities and correlations. These ranges give some indication about the uncertainty in the statistical estimation. For details, see the discussion in the [MarketData](https://github.com/sschlenkrich/MarketData) repository.

### Volatility Data

We normalise volatility data as annualised volatility and measure reference rate terms in years.

In [ ]:
file_name = "data/standard_deviation_30days.csv"
std_table = DataFrame(CSV.File(file_name))
std_table = stack(std_table, 3:size(std_table)[2])
std_table[!, "VOLATILITY"] = std_table[!, "value"] / sqrt(30.0/365.0);
std_table[!, "YEARS"] = std_table[!, "MONTHS"] / 12;

### Correlation Data

In [ ]:
file_name = "data/correlations_30days.csv"
corr_table = DataFrame(CSV.File(file_name))
corr_table = stack(corr_table, 5:size(corr_table)[2])
corr_table[!, "YEARS1"] = Int.(corr_table[!, "MONTHS1"] / 12)
corr_table[!, "YEARS2"] = Int.(corr_table[!, "MONTHS2"] / 12);

## Model Setup

For this analysis, we setup a 3-factor Gaussian HJM model with time-homogenous volatility parameters.

The volatility specification uses zero rate benchmark rates with terms $\{1, 10, 20\}$ (measured in years). We note that the benchmark rate terms deliberately coincide with reference rate terms. This way, we can directly control respective volatilities and correlations.

The model specification is encoded in the `model_params` dictionary below.

In [ ]:
model_params = Dict([
    (("delta_1", ""), 1.00),
    (("delta_2", ""), 10.0),
    (("delta_3", ""), 20.0),
    #
    (("chi_1", ""), 0.01),
    (("chi_2", ""), 0.50),
    (("chi_3", ""), 0.80),
    #
    (("EUR_f_1", ""), 0.0060),
    (("EUR_f_2", ""), 0.0068),
    (("EUR_f_3", ""), 0.0071),
    #
    (("EUR_f_1", "EUR_f_2"), 0.55),
    (("EUR_f_2", "EUR_f_3"), 0.95),
    (("EUR_f_1", "EUR_f_3"), 0.45),
    #
]);

The parameters in the dictionary can be modified. And the impact on model-implied quantities can be observed in below output graphs.

## Model Analysis

In below graphs we compare ranges (box plots) of estimates for historical volatility and correlation with corresponding model-implied quantities.

Model-implied volatilities and correlations are evaluated using the methodology implemented in `DiffFusion.reference_rate_volatility_and_correlation(...)`.

Model-implied volatilities and correlations for 1-year, 10-year and 20-year rates can be controlled effectively by the corresponding input model parameters. Other model-implied quantities are *interpolated* by the model.

In [ ]:
update_rates_plots!(
    [
    ],
    model_params, std_table, corr_table,
    plot_vols = true,
    plot_rates_corrs = true,
    scaling_type = DiffFusion.ZeroRateScaling
)

We find that we can match well the structure of interest rate volatilities for various reference terms. Our tests revealed that it is particularly challenging to model the volatility structure between 1-year and 10-year with the proposed model setup. It turns out that mean reversion parameters can be used to control the volatility structure in that term range. See also the discussion regarding sensitivities in the forthcoming section.

We also find that our model is able to recover the structure of correlations observed from historical time series. The model allows direct control over the correlations `EUR_1__EUR_10`, `EUR_1__EUR_20` and `EUR_10__EUR_20`. All other correlations are *interpolated/extrapolated* from the model specification. In that respect we cannot expect a perfect fit between model-implied correlations and historical correlations. Given that there is also considerable statistical uncertainty about the *true* correlations, we consider the correlation fit reasonable.

If higher accuracy is required in matching reference volatilities and correlations then the number of factors could be increased (e.g. to five) in the model specification.

## Sensitivity Analysis

Calibrating and using the model in practice requires intuition about the model parameters. Sensitivity analysis is an effective tool to gain intuition about model parameters.

We use Automatic Differentiation (AD, via [ForwardDiff.jl](https://github.com/JuliaDiff/ForwardDiff.jl)) to calculate sensitivities of model-implied volatilities and correlations with respect to our model parameters specified in the `model_params` dictionary.

Note that AD sensitivities are calculated as mathematical derivative objects like $\partial f / \partial x$. Model input parameters and reference rate outputs are represented in units of year, percentage or basis points. For a meaningful interpretation of sensitivities the AD derivatives are grouped as follows:

  - Model input parameter type:
    - benchmark tine,
    - mean reversion rates,
    - volatilities,
    - correlations.
  - Reference rate output types:
    - volatilities,
    - correlations.

In [ ]:
plot_model_sensitivities(model_params)

We discuss the sensitivity analysis for the model parameters individually.

As a preceding step, we specify our notation of x- and y-labels in above sensitivity plot.

  - Model input benchmark time parameters are δ_1, δ_2 and δ_3. These parameters are set to 1 year, 10 years and 20 years. The choice is relevant for the interpretation of displayed sensitivities.
  - Model input mean reversion parameters are χ_1, χ_2 and χ_3.
  - Model input volatility parameters are σ_1, σ_2 and σ_3. Recall, that we use `DiffFusion.ZeroRateScaling`. This setting is also relevant for the interpretation of displayed results.
  - Model input correlation parameters are Γ_1,2, Γ_2,3, and Γ_2,3.

  - Model-implied zero rate volatilities are denoted as σ(1y) to σ(20y). The term 1y to 20y denotes the term of the zero rate.
  - Model-implied correlations between zero rates are denoted as Γ(1y, 2y) to Γ(15y, 20y). Again, the term 1y to 20y denotes the term of the zero rate.

### Volatility Parameters

As expected, we find that sensitivity of reference rate volatilities (1/10/20-year terms) with respect to corresponding model parameters is close to one. This represents a consistency check for our model specification and implementation. Note that the sensitivity of the σ(1y) volatility w.r.t. the σ_1 parameter is slightly below 1. This is explained by the impact of mean reversion over the modelled 1-month return horizon.

It is remarkable to see that the σ_2 and σ_3 have considerable impact on the 2-year and 5-year volatilities with opposing signs. This observation is linked to the fact that the modelled zero rates are *overlapping* rates. Such a behaviour needs to be kept in mind when designing automated calibration methodologies to avoid distorted solutions. The shared impact of σ_2 and σ_3 with weights approximately $1/3$ and $2/3$ on the 15-year volatility is more in line with intuition.


Model-implied correlations are mildly affected by volatility parameters. σ_2 and σ_3 volatilities have some impact on 2-year and 5-year correlations, again with opposing signs. Overall, volatility and correlation impacts can reasonably well be disentangled.



### Correlation Parameters

We find again, that the sensitivity of (output correlation) Γ(1y, 20y) with respect to (input correlation) Γ_1,3 and Γ(10y, 20y) with respect to Γ_2,3 is close to one. This is expected given the chosen benchmark times. The sensitivity of Γ(1y, 10y) with respect to Γ_1,2 is considerably below one. This observation is attributed to the interplay with mean reversion.

As expected, other (output) correlations are also impacted by the input correlations. The sensitivity structure is fairly complex. It is advised to be careful when applying automated best-fit calibrations to avoid distorted (or extreme) input correlation parameters.

The Γ_2,3 (10-year vs. 20-year) input correlation also impacts the model-implied volatilities σ(2y) and σ(5y). This sensitivity can be explained by the nature of modelled overlapping zero rates: If correlation 10-year vs. 20-year increases then forward rate volatility 10-year into 20-year also increases. Given that 10-year and 20-year spot rate volatility are fixed (in the setting of sensitivity calculation), the forward rate volatility increase is offset by the 2-year and 5-year spot rate volatility.

### Mean Reversion Parameters

We see that mean reversion particularly impacts volatilities for 2-year and 5-year term. This impact is particularly pronounced for the first mean reversion parameter χ_1. The first mean reversion parameter is the smallest mean reversion parameter and typically close to zero. Other mean reversion parameters are typically considerably larger. A change in the smallest mean reversion parameter has a large impact on the overall mean reversion in the model. Thus, it is expected that it also impacts volatility most. However, χ_1 is intentionally small to model parallel shifts of the yield curve. As a consequence, there is not too much degree of freedom in setting this parameter.

The parameters χ_2 and χ_3 also impact volatilities for 2-year and 5-year term though to a lesser degree. But we can use these parameters to control the slope of volatilities depending on rate term.

Mean reversion also has a considerable impact on correlations. An increase in mean reversion also increases de-correlation. This needs to be kept in mind when using mean reversion to control volatilities. There are side effects to the correlation structure.

### Benchmark Time Parameters

The choice of benchmark times also impacts volatilities and correlations. This impact stems from the fact that the benchmark times specify the terms on the yield curve which are explicitly modelled. A change in benchmark times means a change in *interpolation* of model properties.

The benchmark times are typically set to relevant terms for calibration. This aims at simplifying calibration by other model parameters. Based on this consideration, benchmark times are typically not considered free model parameters for calibration. 

### Calibration Automation

Model calibration procedures can be automated. Given the complex sensitivity structure, a brute-force least-squares optimisation is likely not the method of choice. Instead, calibration objectives and corresponding model parameters should be aligned based on thorough considerations and well understood model properties.